# Mini-Batch K-Means 算法

**核心概念**: Mini-Batch K-Means 是标准 K-Means 的优化变体，通过在小批量数据上迭代更新质心来加速聚类过程

## 算法原理

与标准 K-Means 每次迭代使用全部数据不同，Mini-Batch K-Means 每次迭代仅使用一个随机采样的小批量:

### 算法步骤

1. **初始化**: 随机选择 K 个质心
2. **采样**: 从数据集中随机采样一个小批量 (batch)
3. **分配**: 将小批量中的样本分配给最近的质心
4. **更新**: 使用流式平均更新质心 (考虑历史更新次数)
5. **迭代**: 重复步骤 2-4 直到收敛

### 质心更新公式

$$\mu_i^{(t+1)} = \mu_i^{(t)} + \frac{1}{n_i}(x - \mu_i^{(t)})$$

其中 $n_i$ 是质心 $\mu_i$ 的累计更新次数。

## 优势与适用场景

- **计算效率**: 时间复杂度显著降低，适合大规模数据集
- **内存友好**: 不需要同时加载所有数据到内存
- **在线学习**: 支持增量式学习，可处理流式数据
- **代价**: 结果质量略有损失，但通常可接受

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.datasets import load_iris, make_blobs
from sklearn.metrics import silhouette_score

# 设置随机种子
np.random.seed(42)

# 配置 matplotlib 支持中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

## 1. 基础用法

使用鸢尾花数据集进行演示。

In [ ]:
# 加载数据
iris = load_iris()
X = iris.data
y_true = iris.target

print(f"数据集形状: {X.shape}")

In [ ]:
# 创建 Mini-Batch K-Means 模型
minibatch_kmeans = MiniBatchKMeans(
    n_clusters=3,
    batch_size=32,       # 每批次样本数
    n_init=10,           # 运行次数
    max_iter=100,        # 最大迭代次数
    random_state=42
)

# 拟合并预测
y_pred = minibatch_kmeans.fit_predict(X)

print(f"惯性 (Inertia): {minibatch_kmeans.inertia_:.4f}")
print(f"迭代次数: {minibatch_kmeans.n_iter_}")
print(f"轮廓系数: {silhouette_score(X, y_pred):.4f}")

## 2. 关键参数说明

| 参数 | 说明 | 建议值 |
|------|------|--------|
| `n_clusters` | 簇的数量 | 根据业务需求或评估指标确定 |
| `batch_size` | 每批样本数 | 通常 100-1000，越大越接近标准 K-Means |
| `n_init` | 独立运行次数 | 3-10 |
| `max_iter` | 最大迭代次数 | 100-300 |
| `max_no_improvement` | 早停容忍度 | 10-100 |
| `reassignment_ratio` | 质心重分配阈值 | 0.01 |

In [ ]:
# 探索 batch_size 对结果的影响
batch_sizes = [8, 16, 32, 64, 128]
results = []

for bs in batch_sizes:
    mbk = MiniBatchKMeans(n_clusters=3, batch_size=bs, n_init=10, random_state=42)
    mbk.fit(X)
    score = silhouette_score(X, mbk.labels_)
    results.append({
        'batch_size': bs,
        'inertia': mbk.inertia_,
        'silhouette': score
    })
    print(f"batch_size={bs:3d}: Inertia={mbk.inertia_:.2f}, Silhouette={score:.4f}")

## 3. K-Means vs Mini-Batch K-Means 性能对比

在大规模数据集上比较两种算法的性能差异。

In [ ]:
# 生成大规模合成数据
n_samples = 10000
X_large, y_large = make_blobs(
    n_samples=n_samples,
    centers=5,
    cluster_std=1.0,
    random_state=42
)

print(f"大规模数据集形状: {X_large.shape}")

In [ ]:
# 标准 K-Means
start_time = time.time()
kmeans = KMeans(n_clusters=5, n_init=10, random_state=42)
kmeans.fit(X_large)
kmeans_time = time.time() - start_time
kmeans_silhouette = silhouette_score(X_large, kmeans.labels_)

print("标准 K-Means:")
print(f"  运行时间: {kmeans_time:.4f} 秒")
print(f"  惯性: {kmeans.inertia_:.2f}")
print(f"  轮廓系数: {kmeans_silhouette:.4f}")

# Mini-Batch K-Means
start_time = time.time()
mbkmeans = MiniBatchKMeans(n_clusters=5, batch_size=256, n_init=10, random_state=42)
mbkmeans.fit(X_large)
mbkmeans_time = time.time() - start_time
mbkmeans_silhouette = silhouette_score(X_large, mbkmeans.labels_)

print("\nMini-Batch K-Means:")
print(f"  运行时间: {mbkmeans_time:.4f} 秒")
print(f"  惯性: {mbkmeans.inertia_:.2f}")
print(f"  轮廓系数: {mbkmeans_silhouette:.4f}")

print(f"\n加速比: {kmeans_time / mbkmeans_time:.2f}x")
print(f"质量损失: {(kmeans_silhouette - mbkmeans_silhouette) / kmeans_silhouette * 100:.2f}%")

In [ ]:
# 可扩展性测试: 不同数据规模下的性能
sample_sizes = [1000, 5000, 10000, 20000, 50000]
kmeans_times = []
mbkmeans_times = []

for n in sample_sizes:
    X_test, _ = make_blobs(n_samples=n, centers=5, random_state=42)
    
    # 标准 K-Means
    start = time.time()
    KMeans(n_clusters=5, n_init=3, random_state=42).fit(X_test)
    kmeans_times.append(time.time() - start)
    
    # Mini-Batch K-Means
    start = time.time()
    MiniBatchKMeans(n_clusters=5, batch_size=256, n_init=3, random_state=42).fit(X_test)
    mbkmeans_times.append(time.time() - start)
    
    print(f"n={n:5d}: K-Means={kmeans_times[-1]:.3f}s, Mini-Batch={mbkmeans_times[-1]:.3f}s")

# 可视化
plt.figure(figsize=(10, 6))
plt.plot(sample_sizes, kmeans_times, 'o-', label='K-Means', linewidth=2, markersize=8)
plt.plot(sample_sizes, mbkmeans_times, 's-', label='Mini-Batch K-Means', linewidth=2, markersize=8)
plt.xlabel('样本数量', fontsize=12)
plt.ylabel('运行时间 (秒)', fontsize=12)
plt.title('K-Means vs Mini-Batch K-Means 可扩展性对比', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. 聚类结果可视化

In [ ]:
# 生成二维数据用于可视化
X_2d, y_2d = make_blobs(n_samples=1000, centers=4, cluster_std=0.8, random_state=42)

# K-Means 和 Mini-Batch K-Means 聚类
km = KMeans(n_clusters=4, n_init=10, random_state=42)
mbkm = MiniBatchKMeans(n_clusters=4, batch_size=64, n_init=10, random_state=42)

km_labels = km.fit_predict(X_2d)
mbkm_labels = mbkm.fit_predict(X_2d)

# 可视化对比
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 真实标签
axes[0].scatter(X_2d[:, 0], X_2d[:, 1], c=y_2d, cmap='viridis', s=20, alpha=0.7)
axes[0].set_title('真实标签')
axes[0].set_xlabel('特征 1')
axes[0].set_ylabel('特征 2')

# K-Means
axes[1].scatter(X_2d[:, 0], X_2d[:, 1], c=km_labels, cmap='viridis', s=20, alpha=0.7)
axes[1].scatter(km.cluster_centers_[:, 0], km.cluster_centers_[:, 1],
                c='red', marker='X', s=200, edgecolors='k', linewidths=2)
axes[1].set_title(f'K-Means (Silhouette: {silhouette_score(X_2d, km_labels):.3f})')
axes[1].set_xlabel('特征 1')
axes[1].set_ylabel('特征 2')

# Mini-Batch K-Means
axes[2].scatter(X_2d[:, 0], X_2d[:, 1], c=mbkm_labels, cmap='viridis', s=20, alpha=0.7)
axes[2].scatter(mbkm.cluster_centers_[:, 0], mbkm.cluster_centers_[:, 1],
                c='red', marker='X', s=200, edgecolors='k', linewidths=2)
axes[2].set_title(f'Mini-Batch K-Means (Silhouette: {silhouette_score(X_2d, mbkm_labels):.3f})')
axes[2].set_xlabel('特征 1')
axes[2].set_ylabel('特征 2')

plt.tight_layout()
plt.show()

## 5. 增量学习 (partial_fit)

Mini-Batch K-Means 支持增量学习，可以分批次处理数据，适合流式数据场景。

In [ ]:
# 模拟流式数据场景
X_stream, _ = make_blobs(n_samples=5000, centers=5, random_state=42)

# 创建增量学习模型
mbkm_partial = MiniBatchKMeans(n_clusters=5, random_state=42)

# 分批次训练
batch_size = 500
n_batches = len(X_stream) // batch_size

for i in range(n_batches):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch = X_stream[start_idx:end_idx]
    
    # 增量更新
    mbkm_partial.partial_fit(batch)
    
    if (i + 1) % 2 == 0:
        # 评估当前模型质量
        labels = mbkm_partial.predict(X_stream)
        score = silhouette_score(X_stream, labels)
        print(f"批次 {i+1}/{n_batches}: 轮廓系数 = {score:.4f}")

# 最终结果
final_labels = mbkm_partial.predict(X_stream)
print(f"\n最终轮廓系数: {silhouette_score(X_stream, final_labels):.4f}")

## 总结

### Mini-Batch K-Means 优点

1. **高效性**: 在大规模数据上比标准 K-Means 快数倍至数十倍
2. **内存效率**: 每次仅加载一小批数据，内存占用低
3. **增量学习**: 支持 `partial_fit`，适合流式数据和在线学习场景
4. **质量可接受**: 聚类质量通常与标准 K-Means 相差不大

### 使用建议

- 当数据量超过 10,000 样本时，考虑使用 Mini-Batch K-Means
- `batch_size` 通常设置为 100-1000，需要在速度和质量间权衡
- 对于流式数据，使用 `partial_fit` 进行增量更新
- 如果对聚类质量要求极高，仍应使用标准 K-Means

### 与标准 K-Means 的选择

| 场景 | 推荐算法 |
|------|----------|
| 小数据集 (< 10K) | 标准 K-Means |
| 大数据集 (> 10K) | Mini-Batch K-Means |
| 流式数据 | Mini-Batch K-Means (partial_fit) |
| 质量优先 | 标准 K-Means |
| 速度优先 | Mini-Batch K-Means |